In [ ]:
## IMPORT LIBRERIE ##

import requests
from requests.auth import HTTPBasicAuth
import pandas as pd
import numpy as np
from operator import itemgetter


In [ ]:
## DEFINIZIONE VARIABILI ##

# CREDENZIALI
username_ = 'Di Febo FDS'
pwd_ = 'UaSe5N6g&eja5)['

# FISSE PER IL TEST
competition_to_search = 'SERIE A'
season_to_search = 2022

# VARIABILI ANCHE PER IL TEST
player_to_search = 'LEAO'
team_to_search = 'MILAN'
opponent_to_search = 'INTER'
homeAwayFlag = 'HOME'

if homeAwayFlag == 'HOME':
    homeTeam = team_to_search
    awayTeam = opponent_to_search
else:
    awayTeam = team_to_search
    homeTeam = opponent_to_search

competition_dict = {'SERIE A':'2'}

In [ ]:
## TROVA IL MATCH ID ##

link = 'http://server.sics.it/livefeeds/x16.json?seasonId='+str(season_to_search)+'&competitionId='+competition_dict[competition_to_search]+'&lang=it'

# JSON_data
# Making a get request
match_json_data = requests.get(link, auth = HTTPBasicAuth(username_, pwd_)).json()

#match_json_data
match_json_data['matchData']
match_row = list(filter(lambda a : (a['homeTeam']['name'] == homeTeam) and (a['awayTeam']['name'] == awayTeam), match_json_data['matchData']))[0]

matchId = match_row['id']

print('MatchId: '+str(matchId))

In [ ]:
## TROVA IL TEAM ID ##

if homeAwayFlag == 'HOME':
    teamId = match_row['homeTeam']['id']
else:
    teamId = match_row['awayTeam']['id']

print('TeamId: '+str(teamId))

In [ ]:
## TROVA IL PLAYER ID ##

player_link = ' http://server.sics.it/livefeeds/s02.json?seasonId='+str(season_to_search)+'&teamId='+str(teamId)+'&lang=it'
player_json_data = requests.get(player_link, auth = HTTPBasicAuth(username_, pwd_)).json()

player_json_data
player_row = list(filter(lambda row: row['lastName'] == player_to_search, player_json_data['player']))[0]
playerId = player_row['id']

print('PlayerId: '+str(playerId))

In [ ]:
## EXTRA: APRI URL FOTO GIOCATORE ##

import webbrowser

url = player_row['photo']

#webbrowser.open(url)

In [ ]:
## TROVA TUTTE LE PARTITE GIOCATE DAL PLAYER ID ##

games_player_link = ' http://server.sics.it/livefeeds/s03.json?seasonId='+str(season_to_search)+'&competitionId='+str(competition_dict[competition_to_search])+'&playerId='+str(playerId)+'&lang=it'
games_player_json_data = requests.get(games_player_link, auth = HTTPBasicAuth(username_, pwd_)).json()

games_player_json_data['matchData'][:2]

In [ ]:
## TROVA ALTRI PLAYER ID NELLO STESSO MATCH ID (DIVISO PER TEAM ID) ##

link = 'http://server.sics.it/livefeeds/x17.json?matchId='+str(matchId)+'&lang=it'

# JSON_dataMaking a get request
event_json_data = requests.get(link, auth = HTTPBasicAuth(username_, pwd_)).json()

#event_json_data.keys()
event_json_data['matchData'].keys()

if homeAwayFlag == 'HOME':
    team_player_ids = list(map(lambda row: row['id'], event_json_data['matchData']['homeTeam']['player']))
    opp_player_ids = list(map(lambda row: row['id'], event_json_data['matchData']['awayTeam']['player']))
else:
    team_player_ids = list(map(lambda row: row['id'], event_json_data['matchData']['awayTeam']['player']))
    opp_player_ids = list(map(lambda row: row['id'], event_json_data['matchData']['homeTeam']['player']))

team_player_ids

In [ ]:
# TEST: TROVA VARIABILI PER PLAYER ID NEL MATCH ID ##

player_match_link = ' http://server.sics.it/livefeeds/s04.json?fixtureId='+str(matchId)+'&playerId='+str(playerId)+'&lang=it'
player_match_json_data = requests.get(player_match_link, auth = HTTPBasicAuth(username_, pwd_)).json()

#player_match_json_data
player_match_stats = player_match_json_data['playerStats']['fundamental']
descriptions_test = list(map(lambda c : c['description'], player_match_stats))

player_match_stats[:3]

In [ ]:
descriptions_test[:3]

In [ ]:
## TROVA VARIABILI PER PLAYER ID DELLO STESSO TEAM ID NEL MATCH ID ##

def mergeTwoListsNoDuplicates(first_list,second_list):
    
    in_first = set(first_list)
    in_second = set(second_list)

    in_second_but_not_in_first = in_second - in_first

    result = first_list + list(in_second_but_not_in_first)

    return result
    
team_player_stats = {}

descriptions_list = []

for row_playerId in team_player_ids:
    
    # EVENTS PER PLAYER PER GAME
    team_player_stats[row_playerId] = {}
    player_match_link = ' http://server.sics.it/livefeeds/s04.json?fixtureId='+str(matchId)+'&playerId='+str(row_playerId)+'&lang=it'
    player_match_json_data = requests.get(player_match_link, auth = HTTPBasicAuth(username_, pwd_)).json()
    
    #player_json_data
    player_match_stats = player_match_json_data['playerStats']['fundamental']
    descriptions = list(map(lambda c : c['description'], player_match_stats))
    descriptions_list = mergeTwoListsNoDuplicates(descriptions_list,descriptions)
    
    for desc in descriptions:
        team_player_stats[row_playerId][desc] = float(list(filter(lambda a : a['description'] == desc, player_match_stats))[0]['value'])
        

In [ ]:
team_player_stats[playerId]
#team_player_stats[16254]

In [ ]:
descriptions_list

In [ ]:
## SCEGLIAMO VARIABILI SU CUI AZIONARE GLI INSIGHT ##

chosen_vars = {'Duelli':['Duelli Vinti','Duelli','Duelli Tackle','Duelli Persi Meta Campo Offensiva','Duelli Vinti Meta Campo Offensiva','Palle Recuperate','Palle Recuperate Meta Campo Avversaria'],
               
               'Finalizzazione':['Tiri','Tiri in Porta','Tiri Gol','Tiri di Testa','Tiri da Fuori','Assist','Assist Vincenti','Assist in Area','In Fuorigioco'],
               
               'Gioco':['Passaggi','Passaggi Riusciti','Triangolazioni','Triangolazioni Fatte','Palle Perse','Palle Perse Propria Meta Campo','Giochi Portiere Ricevuti','Giochi Portiere Ricevuti Meta Campo Avversaria','Passaggi Chiave','Passaggi Chiave Ricevuti in Area','Dribbling','Dribbling Positivo','Third Pass','Third Pass Vincenti','Lanci','Lanci Positivi'],
               
               'Palle Laterali':['Cut Back','Cut Back Riusciti','Cross','Cross Riusciti','Palle Laterali']}


In [ ]:
## CREIAMO VARIABILI COMPLESSE SU CUI AZIONARE GLI INSIGHT ##

def createNewMetric(num,den,lista):
    
    try:
        num = float(lista[num])
    except:
        num = 0.0
        
    try:
        den = float(lista[den])
    except:
        den = 0.0

    if den == 0.0:
        return None
    else:
        return round(100*(num/den),1)

var_list_to_add= [['passRiusciti_PERC','Passaggi Riusciti','Passaggi'],
                  ['duelliVinti_PERC','Duelli Vinti','Duelli'],
                  ['duelliAerei_PERC','Duelli Aerei Vinti','Duelli Aerei'],
                 ]

chosen_vars['Percentuali'] = []

for var_row in var_list_to_add:
    new_var = createNewMetric(var_row[1],var_row[2],team_player_stats[playerId])
    if  new_var != None:
        team_player_stats[playerId][var_row[0]] = new_var
    else:
        pass
        
    chosen_vars['Percentuali'].append(var_row[0])

team_player_stats[playerId]

In [ ]:
chosen_vars

In [ ]:
## TROVA I DATI STAGIONALI PER PLAYER ID ##

player_link = ' http://server.sics.it/livefeeds/s05.json?playerId='+str(playerId)+'&lang=it'
seasonal_player_json_data = requests.get(player_link, auth = HTTPBasicAuth(username_, pwd_)).json()
seasonal_fundamental_stats = seasonal_player_json_data['seasonPlayerStats']['fundamental']

seasonal_fundamental_stats[:7]

In [ ]:
## SEPARA LE MEDIE STAGIONALI PER PLAYER ID ##

player_means = list(filter(lambda d : '_MEAN' in str(d['code']), seasonal_fundamental_stats))
player_means[:5]

In [ ]:
## SEPARA LE PERCENTUALI STAGIONALI PER PLAYER ID ##

player_percentages = list(filter(lambda d : '_PERC' in str(d['code']), seasonal_fundamental_stats))
player_percentages[:5]

In [ ]:
def findInsightBasedOnSeasonal_(match_value, avg_season_value):

    if avg_season_value > 0:
        return float(np.round(100*((match_value-avg_season_value)/avg_season_value),1))
    else:
        return -9999

def trySingleDescForPlayer(player_list, desc):

    if desc in player_list.keys():
        return player_list[desc]
    else:
        return 0.0

def positionForDescAmongOtherPlayers(desc,match_value_,list_to_compare):
    
    values_for_team_players = list(map(lambda playerId : trySingleDescForPlayer(team_player_stats[playerId],desc), team_player_ids))

    higher_than_match_value = list(filter(lambda value: value > match_value_, values_for_team_players))
    
    return len(higher_than_match_value)+1


In [ ]:
## CALCOLA VALORI PER GLI INSIGHT ##

insight_based_on_seasonal = []

for main_desc in chosen_vars.keys():
    
    for desc in chosen_vars[main_desc]:
        
        if desc in team_player_stats[playerId].keys():
            match_value_ = float(team_player_stats[playerId][desc])
        else:
            match_value_ = 0.0

        #print(match_value_)

        position_among_team_players = positionForDescAmongOtherPlayers(desc,match_value_,team_player_ids)

        #print(position_among_team_players)

        if 'PERC' in desc:
            try:
                avg_season_value_ = list(filter(lambda a : a['code'] == desc, player_percentages))[0]['value']
                float_avg_season_value_ =  float(avg_season_value_.replace('%',''))
            except:
                float_avg_season_value_ = 0.0
        else:
            try:
                float_avg_season_value_ = float(list(filter(lambda a : a['description'] == desc, player_means))[0]['value'])
            except:
                float_avg_season_value_ = 0.0
    
        #print(float_avg_season_value_)
    
        insight_based_on_seasonal.append([main_desc,desc,match_value_,float_avg_season_value_,position_among_team_players,
                                          findInsightBasedOnSeasonal_(match_value_,float_avg_season_value_)])
    
        #print(insight_based_on_seasonal)
   

In [ ]:
insight_based_on_seasonal

In [ ]:
## CREAZIONE INSIGHT TESTUALI ##

description_index = 1
match_value_index = 2
average_index = 3
position_among_team_players_index = 4
match_value_wrt_average_index = -1

def perc_sign(perc_variation):

    if perc_variation < 0:
        return str(perc_variation)
    else:
        return '+'+str(perc_variation)
        
for main_desc in chosen_vars.keys():

    print ('INSIGHT - '+str(main_desc))
    print (' ')
    rows = list(filter(lambda a : (a[0] == main_desc), insight_based_on_seasonal))

    tops = sorted(list(filter(lambda a : (a[match_value_wrt_average_index] > 5) and (a[match_value_index] > 1), rows)), key = itemgetter(match_value_wrt_average_index), reverse = True)

    print('SOPRA LA MEDIA')
    print (' ')
    if len(tops) > 0:
        
        for single_row in tops:
            print(single_row[description_index]+' : '+str(single_row[match_value_index])
                  +' ('+perc_sign(single_row[match_value_wrt_average_index])+'%) rispetto alla media di '+str(single_row[average_index])+", "
                 +str(single_row[position_among_team_players_index])+"° in squadra"
                 )
    
        print (' ')
    
    else:
        print('Nessuno')
        print(' ')
        
    flops = sorted(list(filter(lambda a : (a[match_value_wrt_average_index] < -5) and (a[match_value_wrt_average_index] > -9999), rows)), key = itemgetter(match_value_wrt_average_index))
    print('SOTTO LA MEDIA')
    print (' ')
    
    if len(flops) > 0:
        
        for single_row in flops:
            print(single_row[description_index]+' : '+str(single_row[match_value_index])+' ('+perc_sign(single_row[match_value_wrt_average_index])+'%) rispetto alla media di '+str(single_row[average_index])+", "
                 +str(single_row[position_among_team_players_index])+"° in squadra"
                 )
        print (' ')
    
    else:
        print('Nessuno')
        print (' ')


In [ ]:
def avoidStringError(string):

    return str(string).replace('\n',' ')

def takeDescriptionFromList(item__,sub_item):

    try:
        new_list = list(map(lambda a : a[sub_item], item__))
        if len(new_list) == 1:
            return avoidStringError(new_list[0])
        else:
            return list(map(lambda d : avoidStringError(d), new_list))
    except:
        return None

def takeDescriptionFromDict(item__,sub_item):

    try:
        return avoidStringError(item__[sub_item])
    except:
        return None

def splitXY(item__,index_pos):

    try:
        return avoidStringError(str(item__).split(';')[index_pos])
    except:
        return None

event_json_data.keys()

In [ ]:
## TROVA SINGOLI EVENTI PER MATCH ID ##

df = pd.DataFrame(event_json_data['event'])

df.head()

In [ ]:
## RIFINISCI DATAFRAME EVENTI ##

df['team_id'] = df.apply(lambda row: takeDescriptionFromDict(row.team, 'id'), axis = 1)
df['team_name'] = df.apply(lambda row: takeDescriptionFromDict(row.team, 'name'), axis = 1)

df['event'] = df.apply(lambda row: takeDescriptionFromDict(row.event, 'description'), axis = 1)
df['tag'] = df.apply(lambda row: takeDescriptionFromList(row.tag, 'description'), axis = 1)
df['period'] = df.apply(lambda row: takeDescriptionFromDict(row.period, 'description'), axis = 1)

df['player_id'] = df.apply(lambda row: takeDescriptionFromList(row.playerFrom, 'id'), axis = 1)
df['player_first_name'] = df.apply(lambda row: takeDescriptionFromList(row.playerFrom, 'firstName'), axis = 1)
df['player_last_name'] = df.apply(lambda row: takeDescriptionFromList(row.playerFrom, 'lastName'), axis = 1)
df['player_name'] = df['player_first_name']+' '+ df['player_last_name']

df['player_to_id'] = df.apply(lambda row: takeDescriptionFromDict(row.playerTo, 'id'), axis = 1)
df['player_to_first_name'] = df.apply(lambda row: takeDescriptionFromDict(row.playerTo, 'firstName'), axis = 1)
df['player_to_last_name'] = df.apply(lambda row: takeDescriptionFromDict(row.playerTo, 'lastName'), axis = 1)
df['player_to_name'] = df['player_to_first_name']+' '+ df['player_to_last_name']

df['start_x'] = df.apply(lambda row: splitXY(row.startPointNorm, 0), axis = 1)
df['start_y'] = df.apply(lambda row: splitXY(row.startPointNorm, 1), axis = 1)

df['end_x'] = df.apply(lambda row: splitXY(row.endPointNorm, 0), axis = 1)
df['end_y'] = df.apply(lambda row: splitXY(row.endPointNorm, 1), axis = 1)

df = df.drop(columns = ['team',
                        'player_first_name','player_last_name','playerFrom',
                        'player_to_first_name','player_to_last_name','playerTo',
                        'startPoint','endPoint','startPointNorm','endPointNorm',
                        'intersectionPoint','intersectionPointNorm'])

df.head()

In [ ]:
## FILTRA SOLO EVENTI RIGUARDANTI PLAYER ID ##

df_playered = df[~df['player_name'].isna()]
df_playered_to = df[~df['player_to_name'].isna()]

player_stats_in_the_game = df_playered[df_playered['player_name'].str.contains(player_to_search)]
player_to_stats_in_the_game = df_playered_to[df_playered_to['player_to_name'].str.contains(player_to_search)]

player_from = player_stats_in_the_game[['event','player_name']].groupby(by = ['event']).count()
player_to = player_to_stats_in_the_game[['event','player_name','player_to_name']].groupby(by = ['event','player_name']).count()

#player_from = player_from[player_name.str.contains(player_to_search)]
#player_to = player_to[player_to_name.str.contains(player_to_search)]

In [ ]:
player_from

In [ ]:
player_to